In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",
               "Felipe Pizarro Marquez", ]
__email__ = ["jose@marroquin.cl", "fpizarro92@live.cl", ]
__status__ = "Development"

## Simulate Image Sets for Photogrammetry

With these functions, it is possible to generate **image sets to be used with a photogrammetry software**.

[blendjupyter.ipynb](blendjupyter.ipynb) is a shared Jupyter module &#171;written to set up and manipulate cameras (`'bpy.types.Camera'`), scenes (`'bpy.context.scene'`)&#187;, among other things.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import blendjupyter as blendj
import math
import os
import bpy

In [ ]:
def range_float(a, b, step):
    """Emulates the np.arange(a, b, step) function of the numpy Python
    module and returns the result as a list.
    
    With this function, IT IS NOT NECESSARY to install numpy for
    Blender."""
    l = list()
    if a > b:
        while a > b:
            l.append(a)
            a -= step
    else:
        if a < b:
            while a < b:
                l.append(a)
                a += step
    return l

In [ ]:
def walk_around(**kwargs):
    """Takes photos to an object with a static camera in z-axis and
    y-axis, and the object rotating from it location (keyword argument
    'move' is 'OBJECT'); or with a camera that walks in different
    heights around of a static object ('move' is 'CAM' or 'SPIRAL').
    Returns a list with (x, y, z) coordinates and rotations (tuples).
    
    >>> l = walk_around(move='CAM')
    >>> l
    [((6.928203230275509, 0.0, 4.000000000000001),
      (60, 0, 90)),
     ...,
     ((4.499377795547838, -1.8637033051562721, 6.346826722329881),
      (37.5, 0, 427.5))]
    """
    r = kwargs.pop('r', 8)  # The radius in 'NONE' units.
    hsteps = kwargs.pop('hsteps', 16) # Number of photos during
                                      # movement in xy-plane.
    vsteps = kwargs.pop('vsteps', 4)
    z0 = kwargs.pop('z0', 30)  # Starting angle for the camera respect
                               # to z-Axis measured from the xy-plane.
    ze = kwargs.pop('ze', 60)  # Ending angle respect to z-Axis.
    # In Blender, a simple way to find out the phase change angle is
    # placing the camera (bpy.types.Camera) in the first (x, y, z)
    # coordinate and manually changing the Y rotation angle of
    # 'XYZ' rotation order.
    dphase = kwargs.pop('dphase', 90)
    move = kwargs.pop('move', 'OBJECT')
    loccam = kwargs.pop('loccam', (0, 0, 0))  # Initial location of
                                              # cam.
    # Theta is the angle that rotates around the z-Axis.
    theta_step = 360 / float(hsteps)
    range_theta = range_float(0, 360, theta_step)
    range_theta = range_theta[:hsteps]
    # delta (float) is a factor to increase or decrease the
    # distance between turns of the movement type 'SPIRAL'.
    delta = kwargs.pop('delta', 1)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(walk_around.__name__,
                  list(kwargs.keys())[-1]))
    # Phi is the angle measured from the z-Axis to the xy-plane.
    phi_init = 90 - z0
    phi_end = 90 - ze
    phi_step = abs(phi_init - phi_end) / float(vsteps)
    l_coord = list()
    if move.lower() == 'cam':
        range_phi = range_float(phi_init, phi_end, phi_step)
    elif move.lower() == 'object' or move.lower() == 'spiral':
        range_phi = range_float(90, phi_end, phi_step)
    range_phi = range_phi[:vsteps]
    x_cam = loccam[0]  # First location for spiral movement.
    for phi in range_phi:
        for theta in range_theta:
            if move.lower() == 'cam':
                # Parameterization of a sphere [1].
                #
                # [1] Carmen, M. (2007). Parametrizaciones. Retrieved
                # from: 
                # http://www.dm.uba.ar/materias/complementos_analisis_Mae/2007/2/parametrizaciones.pdf
                x = r * math.cos(math.radians(theta)) * math.sin(math.radians(phi))
                y = r * math.sin(math.radians(theta)) * math.sin(math.radians(phi))
                z = r * math.cos(math.radians(phi)) + loccam[2]
                rotx = phi
                roty = 0
                rotz = theta + dphase
                l_coord.append(((x, y, z), (rotx, roty, rotz)))
            elif move.lower() == 'object':
                x_cam = loccam[2] * math.cos(math.radians(phi))
                x_cam += loccam[0]
                roty_axis = phi
                rotz_mesh = theta
                l_coord.append(((0, roty_axis, 0),
                                (0, 0, rotz_mesh),
                                (x_cam, loccam[1], loccam[2])))
            elif move.lower() == 'spiral':
                x_cam += math.radians(theta) / delta
                roty_axis = phi
                rotz_mesh = theta
                l_coord.append(((0, roty_axis, 0),
                                (0, 0, rotz_mesh),
                                (x_cam, loccam[1], loccam[2])))
    return l_coord

In [ ]:
def point_shoot_cam(cam, sc, axis, mesh, l_loc_rot, **kwargs):
    """Locates a camera (cam, 'CAMERA') and the object to be
    photographed, points the camera and shoots, and returns the path
    route in which the photos were stored.
    
    Args:
        cam: A 'CAMERA' to point and shoot.
        sc: A Blender scene.
        axis: An axis in Blender 'PLANE_AXIS'.
        mesh: An mesh to be moved and rotated.
        l_loc_rot = A list with rotation (tuple) and location (tuple).
    """
    inrad = kwargs.pop('inrad', False)  # It is True if the rotation
                                        # angles are in radians.  
    move = kwargs.pop('move', 'OBJECT')
    # If onebyone (bool) is True, each photo will be stored alone in a
    # different folder.
    onebyone = kwargs.pop('onebyone', False)
    outdirname = kwargs.pop('outdirname', 'phg-set-0001')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(point_shoot_cam.__name__,
                  list(kwargs.keys())[-1]))
    if onebyone:
        outdirname = 'ss-blend-0001'
    path = blendj.find_out_dir(dirname=outdirname,
                               parentdir='scanner')
    len_llocrot = len(l_loc_rot)
    for i in range(len_llocrot):
        if move.lower() == 'cam':
            loc, rot = l_loc_rot[i]
            cam.location = loc
            if not inrad:
                for j in range(3):  # 3 by each rotation.
                    cam.rotation_euler[j] = math.radians(rot[j])
        elif move.lower() == 'object' or move.lower() == 'spiral':
            axis_rot, mesh_rot, cam_loc = l_loc_rot[i]
            if not inrad:
                for j in range(3):  # 3 by each rotation.
                    axis.rotation_euler[j] = math.radians(axis_rot[j])
                    mesh.rotation_euler[j] = math.radians(mesh_rot[j])
            cam.location = cam_loc
        blendj.shoot_cam(cam,
                         sc,
                         i,
                         len_llocrot,
                         path,
                         prefix='view_',
                         separatedir=onebyone)
    return path

In [ ]:
def take_phg_photos(**kwargs):
    """Prepares a scene and a cam to capture photos of a mesh.
    
    Returns the route of the output folder."""
    meshes = {
        'antique': 'AntiqueBust.stl',
        'apollo': 'ApolloBust.stl',
        'artemis': 'ArtemisStatue.stl',
        'athena': 'Athena.stl',
        'fish': 'FishVase.stl'
        }  # Meshes inside of stl-meshes-phg-cc folder. REMEMBER
           # UNCOMPRESS stl-meshes-phg-cc.zip
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'superscanner-software-s3'))
    blend = kwargs.pop('blend',
                os.path.join(s3path, 'blend-meshes',
                             'ss-ph-studio.blend'))  # File by open.
    dirname = kwargs.pop('dirname',
                  'blend-phg-set-0001')  # Output folder's name.
    # When ssproj (bool) is True, the photos will be stored alone in
    # different folders within the dirname (str) directory created by
    # point_shoot_cam().
    ssproj = kwargs.pop('ssproj', False)
    # Size for the render in (width, height) pixels.
    size = kwargs.pop('size', (2560, 1920))
    scale = kwargs.pop('scale', 1)  # Scale of the scene
                                    # ('bpy.context.scene').
    gray = kwargs.pop('gray', False)
    # movetype (str) can be 'OBJECT', 'CAM' or 'SPIRAL'. See
    # walk_around() (list).
    movetype = kwargs.pop('movetype', 'OBJECT')
    radius = kwargs.pop('radius', 12)
    z0angle = kwargs.pop('z0angle', 0)
    zeangle = kwargs.pop('zeangle', 10)
    zsteps = kwargs.pop('zsteps', 3)
    xysteps = kwargs.pop('xysteps', 12)
    camloc0 = kwargs.pop('camloc0', (2, 0, 9))
    transparent = kwargs.pop('transparent', False)
    extension = kwargs.pop('extension', 'JPEG')
    # deltaturn (float) affects the distance between turns in spiral
    # movement (movetype equals to 'SPIRAL'). The factor that
    # muliplies to xysteps (int) (default value) was determinated
    # empirically.
    deltaturn = kwargs.pop('deltaturn', xysteps ** 1.5)
    mesh = kwargs.pop('mesh', 'fish')
    meshscale = kwargs.pop('meshscale', 0.04)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(take_phg_photos.__name__, list(kwargs.keys())[-1]))
    # Try to open the blend file (blendj, str).
    blendj.open_blend(blend)
    if mesh in meshes:
        mesh = os.path.join(s3path, 'blend-meshes',
                            'stl-meshes-phg-cc', meshes[mesh])
    # Import mesh (str) into opened blend file.
    blendj.put_mesh(mesh, [(0, 0, 0)], scale=meshscale)
    meshname = os.path.basename(mesh).split('.')[0]
    # Get the axis ('EMPTY') associated to the mesh ('MESH') with the
    # name meshname (str).
    axis, mesh = blendj.get_mesh(meshname)
    scene = blendj.set_up_sc(wpx=size[0], hpx=size[1],
                             scalepercent=scale * 100,
                             bw=gray, transp=transparent,
                             ext=extension)
    cam = blendj.get_camera()
    blendj.set_up_cam(cam, loc=camloc0)
    if movetype.lower() == 'cam':
        l_coords = walk_around(move=movetype, r=radius, z0=z0angle,
                               ze=zeangle, hsteps=xysteps,
                               vsteps=zsteps, loccam=cam.location)
    elif movetype.lower() == 'object':
        l_coords = walk_around(move=movetype, z0=z0angle, ze=zeangle,
                               hsteps=xysteps, vsteps=zsteps,
                               loccam=cam.location)
    elif movetype.lower() == 'spiral':
        l_coords = walk_around(move=movetype, z0=z0angle, ze=zeangle,
                               hsteps=xysteps, vsteps=zsteps,
                               loccam=cam.location, delta=deltaturn)
    return point_shoot_cam(cam, scene, axis, mesh, l_coords,
                           move=movetype, onebyone=ssproj,
                           outdirname=dirname)